In [3]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

understanding data
https://www.kaggle.com/c/sf-crime 



In [7]:
#read data
df_sf_train = pd.read_csv('datasets/SFCrime_train.csv')
df_sf_test = pd.read_csv('datasets/SFCrime_test.csv')

In [8]:
df_sf_train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [9]:
df_sf_test.tail()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
884257,884257,2003-01-01 00:01:00,Wednesday,MISSION,2600 Block of BRYANT ST,-122.408983,37.751987
884258,884258,2003-01-01 00:01:00,Wednesday,NORTHERN,1900 Block of WASHINGTON ST,-122.425342,37.792681
884259,884259,2003-01-01 00:01:00,Wednesday,INGLESIDE,5500 Block of MISSION ST,-122.445418,37.712075
884260,884260,2003-01-01 00:01:00,Wednesday,BAYVIEW,1500 Block of HUDSON AV,-122.387394,37.739479
884261,884261,2003-01-01 00:01:00,Wednesday,TARAVAL,1500 Block of SLOAT BL,-122.489714,37.733950


In [11]:
#show another info
print('Number of Categories: ', df_sf_train.Category.nunique())
print('Number of PdDistricts: ', df_sf_train.PdDistrict.nunique())
print('Number of DayOfWeeks: ', df_sf_train.DayOfWeek.nunique())
print('_________________________________________________')
# Show some useful Information
df_sf_train.info()

Number of Categories:  39
Number of PdDistricts:  10
Number of DayOfWeeks:  7
_________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


We have 878049 Observations of 9 variables
We have a 'Dates' column which contains the date and time of the occurence of the crime, but it's a String.
We have spatial coordinates ( Latitude and Longitude ) of the exact place of the crime.
The Target column is 'Category', which is a Categorical Column ( 39 categories )
The 'DayOfWeek' column is also Categorical ( 7 days )
The 'PdDistrict' column is also Categorical ( 10 districts )
The 'Address' column indicates whether the crime location was an intersection of two roads
The 'Resolution' column will be droped ( It won't help us with prediction 

In [12]:
df_sf_train = df_sf_train.drop('Resolution', axis=1)

In [13]:
#dealling with Dates 

df_sf_train.Dates.dtype

dtype('O')

In [14]:
#dealling with Dates values and typos
assert df_sf_train.Dates.isnull().any() == False
assert df_sf_test.Dates.isnull().any() == False

In [15]:
assert df_sf_train.Dates.str.match('\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d').all() == True
assert df_sf_test.Dates.str.match('\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d').all() == True

In [16]:
#change all to pandas datatime
df_sf_train['Date'] = pd.to_datetime(df_sf_train.Dates)
df_sf_test['Date'] = pd.to_datetime(df_sf_test.Dates)

train_df = df_sf_train.drop('Dates', axis=1)
test_df = df_sf_test.drop('Dates', axis=1)
train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date
123771,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,INGLESIDE,700 Block of OCEAN AV,-122.448425,37.722862,2013-09-19 08:00:00


In [17]:
# Confirm that it was parsed to Datetime
train_df.Date.dtype

dtype('<M8[ns]')

In [18]:
#ass features crime in Day or night
train_df['IsDay'] = 0
train_df.loc[ (train_df.Date.dt.hour > 6) & (train_df.Date.dt.hour < 20), 'IsDay' ] = 1
test_df['IsDay'] = 0
test_df.loc[ (test_df.Date.dt.hour > 6) & (test_df.Date.dt.hour < 20), 'IsDay' ] = 1

train_df.sample(3)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date,IsDay
324984,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Tuesday,NORTHERN,BUSH ST / LAGUNA ST,-122.428528,37.787650,2010-11-16 18:00:00,1
403942,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Sunday,TARAVAL,4100 Block of LINCOLN WY,-122.502442,37.764242,2009-09-06 12:00:00,1
404124,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Saturday,TENDERLOIN,300 Block of ELLIS ST,-122.411988,37.785023,2009-09-05 23:16:00,0


In [19]:
#encode by days
days_to_int_dic = {
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7,
}
train_df['DayOfWeek'] = train_df['DayOfWeek'].map(days_to_int_dic)
test_df ['DayOfWeek'] = test_df ['DayOfWeek'].map(days_to_int_dic)

train_df.DayOfWeek.unique()

array([3, 2, 1, 7, 6, 5, 4])

In [21]:
#create hour, month, year colomn
train_df['Hour'] = train_df.Date.dt.hour
train_df['Month'] = train_df.Date.dt.month
train_df['Year'] = train_df.Date.dt.year
train_df['Year'] = train_df['Year'] - 2000 # The Algorithm doesn't know the difference. It's just easier to work like that


train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date,IsDay,Hour,Month,Year
224660,ASSAULT,THREATS AGAINST LIFE,1,TARAVAL,1200 Block of 44TH AV,-122.50405,37.763299,2012-05-14 22:26:00,0,22,5,12


In [22]:
#clyclic characteristic oh month and days of weeks

train_df['HourCos'] = np.cos((train_df['Hour']*2*np.pi)/24 )
train_df['DayOfWeekCos'] = np.cos((train_df['DayOfWeek']*2*np.pi)/7 )
train_df['MonthCos'] = np.cos((train_df['Month']*2*np.pi)/12 )

test_df['HourCos'] = np.cos((test_df['Hour']*2*np.pi)/24 )
test_df['DayOfWeekCos'] = np.cos((test_df['DayOfWeek']*2*np.pi)/7 )
test_df['MonthCos'] = np.cos((test_df['Month']*2*np.pi)/12 )

train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date,IsDay,Hour,Month,Year,HourCos,DayOfWeekCos,MonthCos
494516,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",3,SOUTHERN,800 Block of 2ND ST,-122.389744,37.757909,2008-05-28 02:45:00,0,2,5,8,0.866025,-0.900969,-0.866025


In [23]:
#encode category pddistric
train_df = pd.get_dummies(train_df, columns=['PdDistrict'])
test_df  = pd.get_dummies(test_df,  columns=['PdDistrict'])
train_df.sample(2)

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
108771,KIDNAPPING,FALSE IMPRISONMENT,3,2400 Block of 37TH AV,-122.494982,37.741246,2013-11-27 11:15:00,1,11,11,...,0,0,0,0,0,0,0,0,1,0
750171,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",4,6TH ST / NATOMA ST,-122.407693,37.780163,2004-09-16 19:15:00,1,19,9,...,0,0,0,0,0,0,0,1,0,0


In [24]:
#label encode category
from sklearn.preprocessing import LabelEncoder

cat_le = LabelEncoder()
train_df['CategoryInt'] = pd.Series(cat_le.fit_transform(train_df.Category))
train_df.sample(5)
#cat_le.classes_

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt
220998,NON-CRIMINAL,LOST PROPERTY,4,HYDE ST / PACIFIC AV,-122.418295,37.795464,2012-05-31 15:15:00,1,15,5,...,1,0,0,0,0,0,0,0,0,20
361924,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,3,0 Block of ARLETA AV,-122.403663,37.712696,2010-05-05 07:00:00,1,7,5,...,0,1,0,0,0,0,0,0,0,16
805924,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,1,1200 Block of FULTON ST,-122.437369,37.777014,2003-12-22 23:00:00,0,23,12,...,0,0,0,0,1,0,0,0,0,16
271361,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,3,0 Block of TRUMBULL ST,-122.428843,37.730821,2011-09-07 21:00:00,0,21,9,...,0,1,0,0,0,0,0,0,0,16
94886,OTHER OFFENSES,PAROLE VIOLATION,5,6100 Block of CALIFORNIA ST,-122.483417,37.784009,2014-02-07 08:03:00,1,8,2,...,0,0,0,0,0,1,0,0,0,21


In [26]:
train_df['InIntersection'] = 1
train_df.loc[train_df.Address.str.contains('Block'), 'InIntersection'] = 0

test_df['InIntersection'] = 1
test_df.loc[test_df.Address.str.contains('Block'), 'InIntersection'] = 0

train_df.sample(5)

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt,InIntersection
568230,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,4,SPEAR ST / MISSION ST,-122.394065,37.792538,2007-05-03 23:00:00,0,23,5,...,0,0,0,0,0,1,0,0,16,1
445954,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",2,SUNNYDALE AV / RUTLAND ST,-122.408641,37.709846,2009-02-03 14:53:00,1,14,2,...,1,0,0,0,0,0,0,0,21,1
827027,OTHER OFFENSES,RESISTING ARREST,5,1200 Block of DIVISADERO ST,-122.439142,37.781049,2003-09-05 22:05:00,0,22,9,...,0,0,0,1,0,0,0,0,21,0
417067,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",1,INGALLS ST / GEORGE CT,-122.382274,37.730622,2009-07-06 22:04:00,0,22,7,...,0,0,0,0,0,0,0,0,21,1
362897,DRUG/NARCOTIC,POSSESSION OF HALLUCINOGENIC FOR SALES,7,500 Block of JOHNFKENNEDY DR,-122.466205,37.772541,2010-04-25 13:33:00,1,13,4,...,0,0,0,0,1,0,0,0,7,0


In [27]:
#check the columns
train_df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'Address', 'X', 'Y', 'Date',
       'IsDay', 'Hour', 'Month', 'Year', 'HourCos', 'DayOfWeekCos', 'MonthCos',
       'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE',
       'PdDistrict_MISSION', 'PdDistrict_NORTHERN', 'PdDistrict_PARK',
       'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL',
       'PdDistrict_TENDERLOIN', 'CategoryInt', 'InIntersection'],
      dtype='object')

In [28]:
#gettinhg ready for data train

feature_cols = ['X', 'Y', 'IsDay', 'DayOfWeek', 'Month', 'Hour', 'Year', 'InIntersection',
                'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE',
                'PdDistrict_MISSION', 'PdDistrict_NORTHERN', 'PdDistrict_PARK',
                'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN']
target_col = 'CategoryInt'

train_x = train_df[feature_cols]
train_y = train_df[target_col]

test_ids = test_df['Id']
test_x = test_df[feature_cols]

In [29]:
train_x.sample(1)

,X,Y,IsDay,DayOfWeek,Month,Hour,Year,InIntersection,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
852917,-122.416075,37.78445,0,4,5,23,3,1,0,0,0,0,0,0,0,0,0,1


In [30]:
test_x.sample(1)

,X,Y,IsDay,DayOfWeek,Month,Hour,Year,InIntersection,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
412822,-122.421746,37.795486,1,6,8,12,9,0,0,0,0,0,1,0,0,0,0,0


In [35]:
#------------------------------
#**** another way ****
# split data into train and test sets
# seed randomness untuk tracable 
seed = 7 # saran : set a prime number # kurang lebih 1%
test_size = 0.33   #33% sebagai data test, train : test 30:70,20:80, 33:67
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=test_size, random_state=seed)

In [36]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)


/Users/prast/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:27:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [37]:
#--------------------------
#Xboost
type(train_x), type(train_y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [38]:
import xgboost as xgb
train_xgb = xgb.DMatrix(train_x, label=train_y)
test_xgb  = xgb.DMatrix(test_x)

In [39]:
params = {
    'max_depth': 4,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 39,
}

In [40]:
CROSS_VAL = False
if CROSS_VAL:
    print('Doing Cross-validation ...')
    cv = xgb.cv(params, train_xgb, nfold=3, early_stopping_rounds=10, metrics='mlogloss', verbose_eval=True)
    cv

In [41]:
SUBMIT = not CROSS_VAL
if SUBMIT:
    print('Fitting Model ...')
    m = xgb.train(params, train_xgb, 10)
    res = m.predict(test_xgb)
    cols = ['Id'] + cat_le.classes_
    submission = pd.DataFrame(res, columns=cat_le.classes_)
    submission.insert(0, 'Id', test_ids)
    submission.to_csv('submission.csv', index=False)
    print('Done Outputing !')
    print(submission.sample(3))
else:
    print('NOT SUBMITING')

Fitting Model ...
[08:54:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[08:54:56] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Done Outputing !
            Id     ARSON   ASSAULT  BAD CHECKS   BRIBERY  BURGLARY  \
276736  276736  0.004908  0.038547    0.004647  0.004692  0.008330   
369887  369887  0.005446  0.061115    0.004869  0.004822  0.066053   
221922  221922  0.007669  0.095547    0.004921  0.004746  0.025324   

        DISORDERLY CONDUCT  DRIVING UNDER T